In [3]:
import gym
import keras_gym as km
from tensorflow import keras


# MDP
env = gym.make('CartPole-v0')

# policy
pi = km.RandomPolicy(env)


km.render_episode(env, pi, step_delay_ms=50)

In [8]:
class Linear(km.FunctionApproximator):
    def body(self, SA, variable_scope):
        return keras.layers.Flatten()(SA)
    


# Q-function (s,a) -> q(s,a)
func = Linear(env, lr=0.001)
q = km.QTypeI(func, update_strategy='sarsa', gamma=0.9, bootstrap_n=1)

# policy
pi = km.EpsilonGreedy(q, epsilon=0.1)


# learn
k = 0

for ep in range(100):
    # exploration schedule
    pi.epsilon = 0.1 if ep < 10 else 0.01

    # run episode
    s = env.reset()

    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)

        q.update(s, a, r, done)

        if done:
            if t == env.spec.max_episode_steps - 1:
                k += 1
                print("successes: {}".format(k))
            else:
                k = 0
                print("failed after {} steps".format(t))
            break

        s = s_next
        
    if k == 10:
        break

    
km.render_episode(env, pi, step_delay_ms=10)

failed after 20 steps
failed after 8 steps
failed after 10 steps
failed after 10 steps
failed after 10 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 10 steps
failed after 9 steps
failed after 8 steps
failed after 8 steps
failed after 8 steps
failed after 10 steps
failed after 10 steps
failed after 73 steps
failed after 9 steps
failed after 33 steps
failed after 9 steps
failed after 8 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 8 steps
failed after 9 steps
failed after 8 steps
failed after 8 steps
failed after 8 steps
failed after 8 steps
failed after 33 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 10 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 10 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 9 steps
failed after 8 steps
failed after 9 steps
f